In [1]:
import requests
import re
import csv
import lxml.html as lh
import pandas as pd
import numpy as np 

In [2]:
import mysql.connector

In [3]:
mydb = mysql.connector.connect(
  host="ocala.bab-c.com",
  user="pgot",
  password="pgot1",
  database="pgot"
)

mycursor = mydb.cursor()

In [4]:
from bs4 import BeautifulSoup

In [5]:
r=requests.get("https://www.mscorecard.com/mscorecard/courses.php?page=1&CourseName=&Country=USA&SubmitButton=Search", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})

In [6]:
c=r.content
soup=BeautifulSoup(c,"html.parser")

In [7]:
pre_url="https://www.mscorecard.com/mscorecard/courses.php?page="
post_url="&CourseName=&Country=USA&SubmitButton=Search"

In [8]:
page_nr=soup.find_all('select', {"class":"form-control"}) [1]
options = page_nr.find_all("option")
last_page=[y.text for y in options][-1]
first_page=[y.text for y in options][0]
header={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
file = open('GC_course.csv', 'w')
y_file = open('GC_yards.csv', 'w')
writer = csv.writer(file)
y_writer = csv.writer(y_file)
course_id=0

In [9]:
# write title row
writer.writerow(['GC Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Email', 'Website'])
y_writer.writerow(['Level','CID','yards-hole1','yards-hole2','yards-hole3','yards-hole4','yards-hole5','yards-hole6','yards-hole7','yards-hole8','yards-hole9','yards-hole10','yards-hole11','yards-hole12','yards-hole13','yards-hole14','yards-hole15','yards-hole16','yards-hole17','yards-hole18'])

In [10]:
#sql insert course data
sql1 = "INSERT INTO courses (ccname, address, city, state, zip, phone, email, website) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
sql2 = "INSERT INTO gc_tees (level, cid) VALUES (%s, %s)"
sql3 = "INSERT INTO gc_yards (tees, cid, yards_hole0,yards_hole1,yards_hole2,yards_hole3,yards_hole4,yards_hole5,yards_hole6,yards_hole7,yards_hole8,yards_hole9,yards_hole10,yards_hole11,yards_hole12,yards_hole13,yards_hole14,yards_hole15,yards_hole16,yards_hole17) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sql4 = "INSERT INTO gc_data (level, cid, par_hole0,par_hole1,par_hole2,par_hole3,par_hole4,par_hole5,par_hole6,par_hole7,par_hole8,par_hole9,par_hole10,par_hole11,par_hole12,par_hole13,par_hole14,par_hole15,par_hole16,par_hole17) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sql5 = "INSERT INTO gc_handicap (level, cid, han_hole0,han_hole1,han_hole2,han_hole3,han_hole4,han_hole5,han_hole6,han_hole7,han_hole8,han_hole9,han_hole10,han_hole11,han_hole12,han_hole13,han_hole14,han_hole15,han_hole16,han_hole17) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sql6 = "INSERT INTO gc_sloperate (teebox, cid, rate1, slope1) VALUES (%s, %s, %s, %s)"

In [11]:
def gc_course_function(gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_web, gc_email):
    ## sql1 values
    val1 = (gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_web, gc_email)      
    mycursor.execute(sql1,val1)
    
    global course_id
    course_id = mycursor.lastrowid
    mydb.commit()
                
    print(mycursor.rowcount, "GC Course record inserted.")

In [12]:
def gc_tees_function(level,course_id):
    ## sql2 values
    val2 = (level, course_id)      
    mycursor.execute(sql2,val2)
    
    tees_id.append(mycursor.lastrowid)
    
    mydb.commit()
                
    print(mycursor.rowcount, "GC Tee record inserted.")

In [13]:
def gc_yards_function(level,course_id,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17):
    ## sql3 values
    val3 = (level,course_id,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17)      
    mycursor.execute(sql3,val3)
    
    global yards_id
    yards_id = mycursor.lastrowid
    mydb.commit()
                
    print(mycursor.rowcount, "GC Yards record inserted.")

In [14]:
def gc_data_function(level,course_id,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17):
    ## sql4 values
    val4 = (level,course_id,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17) 
    mycursor.execute(sql4,val4)
    
    global par_id
    par_id = mycursor.lastrowid
    mydb.commit()
                
    print(mycursor.rowcount, "GC Par record inserted.")

In [15]:
def gc_handicap_function(level,course_id,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17):
    ## sql5 values
    val5 = (level,course_id,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17) 
    mycursor.execute(sql5,val5)
    
    global han_id
    han_id = mycursor.lastrowid
    mydb.commit()
                
    print(mycursor.rowcount, "GC Hanidcap record inserted.")

In [16]:
def gc_sloperate_function(level,course_id,s,r):
    ## sql6 values
    val6 = (level,course_id,s,r) 
    mycursor.execute(sql6,val6)
    
    mydb.commit()
                
    print(mycursor.rowcount, "GC Slope/Rating record inserted.")

In [17]:
for page in range(int(first_page),int(last_page)+1):
    #print(pre_url+str(page)+post_url)
    gc=(pre_url+str(page)+post_url)
    #print(gc)
    r1=requests.get(gc)
    c1=r1.content
    soup=BeautifulSoup(c1,"html.parser")
    #print(soup.prettify())
    all_gc = [tag['href'] for tag in soup.find_all('a', {'class':"no-hover"})]
    #print(all_gc)
    l=[]
    for item in all_gc:
            df = pd.read_html("https://www.mscorecard.com/mscorecard/"+str(item))[0]
            r2=requests.get("https://www.mscorecard.com/mscorecard/"+str(item))
            c2=r2.content
            #Store the contents of the website under doc
            doc = lh.fromstring(c2)
            #Parse data that are stored between <tr>..</tr> of HTML
            tr_elements = doc.xpath('//tr')
            #Test for the number of holes
            if (len(tr_elements)) == 22:
                h=['2','3','4','5','6','7','8','9','10']
                ms=12
                mcr=13
                ws=14
                wcr=15
            if (len(tr_elements)) == 33:
                h=['2','3','4','5','6','7','8','9','10','12','13','14','15','16','17','18','19','20']
                ms=23
                mcr=24
                ws=25
                wcr=26
            #Create empty list
            tees=[]
            tees_id=[]
            yards=[]
            holes=[]
            col=[]
            par=[]
            w_par=[]
            m_hcp=[]
            w_hcp=[]
            m_slope=[]
            m_rating=[]
            w_slope=[]
            w_rating=[]
            i=0
            j=2
            y=0
            p=0
            wp=0
            #For each row, store each first element (header) and an empty list
            for t in tr_elements[1]:
                i+=1
                name=t.text_content()
                #print ('%d:"%s"'%(i,name))
                #print(name)
                if name not in ['Hole','Par','Hcp']:
                    tees.append(name)
                col.append((name,[]))
            #print('Tees', tees)
            
            #  front 9 and back 9 yardages
            def get_value(index):
                if (len(tr_elements)) == 22:
                    return list(np.concatenate([df.iloc[:,index][2:11].to_numpy()]))
                if (len(tr_elements)) == 33:
                    return list(np.concatenate([df.iloc[:,index][2:11].to_numpy(), df.iloc[:,index][12:21].to_numpy()]))
            yards = []
            for i in range(3,len(tees)+3):
                yards.append(get_value(i))
            #print('Yards', yards) 
            #convert all '-' to zero (0)
            yards = np.array(yards)
            yards[yards == '-'] = 0
            
            # front 9 and back 9 MEN pars and handicaps
            for x in h:
                for t in (tr_elements[int(x)]):
                    name=t.text_content()
                    holes.append(name)
                par.append(holes[1])
                m_hcp.append(holes[2])
                #holes.clear()
                del holes[:]
            #print('Men par', par)
            #convert all '-' to zero (0)
            par = np.array(par)
            par[par == '-'] = 0
            #print('Men Handicap', m_hcp)
            #convert all '-' to zero (0)
            m_hcp = np.array(m_hcp)
            m_hcp[m_hcp == '-'] = 0
            # front 9 and back 9 WOMEN pars and handicaps
            for x in h:
                for t in (tr_elements[int(x)]):
                    name=t.text_content()
                    holes.append(name)
                w_par.append(holes[len(tees)+3])
                w_hcp.append(holes[len(tees)+4])
                #holes.clear()
                del holes[:]
            #print('Women par', w_par)
            #convert all '-' to zero (0)
            w_par = np.array(w_par)
            w_par[w_par == '-'] = 0
            #print('Women Handicap', w_hcp)
            w_hcp = np.array(w_hcp)
            w_hcp[w_hcp == '-'] = 0
            # men slope
            for t in (tr_elements[ms]):
                name=t.text_content()
                holes.append(name)
            for i in range(len(tees)):
                i+=1
                m_slope.append(holes[i])
            del holes[:]
            #convert all '-' to zero (0)
            m_slope = np.array(m_slope)
            m_slope[m_slope == '-'] = 0
            #print('Men slope', m_slope)
            
            # men rating
            for t in (tr_elements[mcr]):
                name=t.text_content()
                holes.append(name)
            for i in range(len(tees)):
                i+=1
                m_rating.append(holes[i])
            del holes[:]
            #convert all '-' to zero (0)
            m_rating = np.array(m_rating)
            m_rating[m_rating == '-'] = 0
            #print('Men rating', m_rating)
            
            # women slope
            for t in (tr_elements[ws]):
                name=t.text_content()
                holes.append(name)
            for i in range(len(tees)):
                i+=1
                w_slope.append(holes[i])
            del holes[:]
            #convert all '-' to zero (0)
            w_slope = np.array(w_slope)
            w_slope[w_slope == '-'] = 0
            #print('Women slope', w_slope)
            
            # women rating
            for t in (tr_elements[wcr]):
                name=t.text_content()
                holes.append(name)
            for i in range(len(tees)):
                i+=1
                w_rating.append(holes[i])
            del holes[:]
            #convert all '-' to zero (0)
            w_rating = np.array(w_rating)
            w_rating[w_rating == '-'] = 0
            #print('Women rating',  w_rating)
            
            soup=BeautifulSoup(c2,"html.parser")
            #print(soup.prettify())
            course = soup.find('div', class_='page-content')
            gc_name = course.h1.text
            addr = soup.find('div', class_='col-md-4')
            innerHtml = addr.decode_contents()
            addr_list =innerHtml.split('<br/>')
            x=re.findall(r'\d+',addr_list[1])
            str1=""
            for ele in x:
                str1 += ele
    
            web = soup.find('div', class_='col-md-8')
            
            ### start here
            gc_addr=addr_list[0]
            gc_city=addr_list[1].replace(str1,'').split(',')[0]
            try:
                gc_state=addr_list[1].replace(str1,'').split(',')[1]
            except:
                gc_state=''
            try:
                gc_zip=str1
            except:
                gc_zip=''
            try:
                gc_phone=web.div.text.replace("phone","")
            except:
                gc_phone=''
            try:
                gc_email=web.select('a[href^=mailto]')
                gc_email=str(gc_email)[1:-1]
            except:
                gc_email=''
            try:
                gc_web=web.select('a[href^=http]')
                gc_web=str(gc_web)[1:-1]
            except:
                gc_web=''
            
            #define the regular expressions for both http and mailto
            regex2 = '<a href="http://(.*?)".*'
            regex = '<a href="mailto:(.*?)".*'
            
            try:
                gc_web = re.match(regex2, gc_web).group(1)
                gc_email = re.match(regex, gc_email).group(1)
            except:
                gc_web= None
                gc_email = None
                x=1
    
            #if gc_web is not None:
            #    print(gc_web)
            #if gc_email is not None:
            #    print(gc_email)
            
            #courses
            gc_course_function(gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_web, gc_email)
            #tees
            for i in tees: 
                gc_tees_function(i, course_id)
            #yards
            while y < len(yards):
                j = 0
                while j < len(yards[0]):
                    #print (tees[y], yards[y])
                    j += 1
                #change tee here
                if (len(yards[0])) == 18:
                    #print (tees_id[y],course_id,yards[y][0],yards[y][1],yards[y][2],yards[y][3],yards[y][4],yards[y][5],yards[y][6],yards[y][7],yards[y][8],yards[y][9],yards[y][10],yards[y][11],yards[y][12],yards[y][13],yards[y][14],yards[y][15],yards[y][16],yards[y][17])
                    gc_yards_function(tees_id[y],course_id,yards[y][0],yards[y][1],yards[y][2],yards[y][3],yards[y][4],yards[y][5],yards[y][6],yards[y][7],yards[y][8],yards[y][9],yards[y][10],yards[y][11],yards[y][12],yards[y][13],yards[y][14],yards[y][15],yards[y][16],yards[y][17])
                    gc_data_function(tees_id[y],course_id,par[0],par[1],par[2],par[3],par[4],par[5],par[6],par[7],par[8],par[9],par[10],par[11],par[12],par[13],par[14],par[15],par[16],par[17])
                    gc_data_function(tees_id[y],course_id,w_par[0],w_par[1],w_par[2],w_par[3],w_par[4],w_par[5],w_par[6],w_par[7],w_par[8],w_par[9],w_par[10],w_par[11],w_par[12],w_par[13],w_par[14],w_par[15],w_par[16],w_par[17])
                    gc_handicap_function(tees_id[y],course_id,w_hcp[0],w_hcp[1],w_hcp[2],w_hcp[3],w_hcp[4],w_hcp[5],w_hcp[6],w_hcp[7],w_hcp[8],w_hcp[9],w_hcp[10],w_hcp[11],w_hcp[12],w_hcp[13],w_hcp[14],w_hcp[15],w_hcp[16],w_hcp[17])
                    gc_handicap_function(tees_id[y],course_id,m_hcp[0],m_hcp[1],m_hcp[2],m_hcp[3],m_hcp[4],m_hcp[5],m_hcp[6],m_hcp[7],m_hcp[8],m_hcp[9],m_hcp[10],m_hcp[11],m_hcp[12],m_hcp[13],m_hcp[14],m_hcp[15],m_hcp[16],m_hcp[17])
                if (len(yards[0])) == 9:
                    gc_yards_function(tees_id[y],course_id,yards[y][0],yards[y][1],yards[y][2],yards[y][3],yards[y][4],yards[y][5],yards[y][6],yards[y][7],yards[y][8],"0","0","0","0","0","0","0","0","0")
                    gc_data_function(tees_id[y],course_id,par[0],par[1],par[2],par[3],par[4],par[5],par[6],par[7],par[8],"0","0","0","0","0","0","0","0","0")
                    gc_data_function(tees_id[y],course_id,w_par[0],w_par[1],w_par[2],w_par[3],w_par[4],w_par[5],w_par[6],w_par[7],w_par[8],"0","0","0","0","0","0","0","0","0")
                    gc_handicap_function(tees_id[y],course_id,m_hcp[0],m_hcp[1],m_hcp[2],m_hcp[3],m_hcp[4],m_hcp[5],m_hcp[6],m_hcp[7],m_hcp[8],"0","0","0","0","0","0","0","0","0")
                    gc_handicap_function(tees_id[y],course_id,w_hcp[0],w_hcp[1],w_hcp[2],w_hcp[3],w_hcp[4],w_hcp[5],w_hcp[6],w_hcp[7],w_hcp[8],"0","0","0","0","0","0","0","0","0")
                gc_sloperate_function(tees_id[y],course_id,m_slope[y],m_rating[y])
                gc_sloperate_function(tees_id[y],course_id,w_slope[y],w_rating[y])
                y += 1        
                
            #print (str(gc_name) + ' ' + str(gc_addr) + ' ' + str(gc_city) + ' ' + str(gc_state) + ' ' + str(gc_zip) + ' ' + str(gc_phone) + ' ' + str(gc_email) + ' ' + str(gc_web))            

            #writer.writerow([gc_name.encode('utf-8'), gc_addr.encode('utf-8'), gc_city.encode('utf-8'), gc_state.encode('utf-8'), gc_zip.encode('utf-8'), gc_phone.encode('utf-8'), gc_email.encode('utf-8'), gc_web.encode('utf-8')])
            writer.writerow([gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_email, gc_web])
            #y_writer.writerow([cid,yards-hole1,yards-hole2,yards-hole3,yards-hole4,yards-hole5,yards-hole6,yards-hole7,yards-hole8,yards-hole9,yards-hole10,yards-hole11,yards-hole12,yards-hole13,yards-hole14,yards-hole15,yards-hole16,yards-hole17,yards-hole18])
   

(1L, 'GC Course record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC 

(1L, 'GC Course record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC 

(1L, 'GC Course record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Tee record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Hanidcap record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Slope/Rating record inserted.')
(1L, 'GC Yards record inserted.')
(1L, 'GC Par record inserted.')
(1L, 'GC 

KeyboardInterrupt: 

In [ ]:
file.close()
y_file.close()